<a href="https://colab.research.google.com/github/MYVirajani/AI-Exam-Evaluation-System/blob/Fine-tunned-model/Finetuning_deepseek_R1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install unsloth

In [2]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.5: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0.05, # Can be set to any, but = 0 is optimized
    bias = "none",    # Can be set to any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3977,
    use_rslora = False,  # unsloth also supports rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.6.5 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [4]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [5]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [6]:
from google.colab import files
uploaded = files.upload()


Saving all_data.json to all_data.json


In [17]:
# import json

# with open("all_data.json", "r") as f:
#     data = json.load(f)

# for entry in data:
#     if isinstance(entry.get("input"), dict):
#         # Convert dict to JSON string to keep all data
#         entry["input"] = json.dumps(entry["input"])
#     elif entry.get("input") is None:
#         entry["input"] = ""

# with open("all_data_fixed.json", "w") as f:
#     json.dump(data, f)

import json

with open("all_data.json", "r") as f:
    data = json.load(f)

fixed_data = []

for entry in data:
    input_data = entry.get("input", {})
    output_data = entry.get("output", {})

    # Extract values with fallback
    question = input_data.get("question", "")
    student_answer = input_data.get("student_answer", "")
    model_answer = input_data.get("model_answer", "")
    knowledge_base = input_data.get("knowledge_base", "")
    max_score = input_data.get("max_score", "")

    # Handle both rubric formats
    rubric = input_data.get("rubric", "")
    if isinstance(rubric, dict):
        rubric_text = f"Criteria: {rubric.get('criteria', '')}\nLevels: {json.dumps(rubric.get('levels', {}))}"
    else:
        rubric_text = rubric  # It's a string already

    # Output section
    feedback = output_data.get("feedback", "")
    score = output_data.get("score", "")

    # Create unified training text
    formatted_text = (
        f"### Instruction:\n{entry.get('instruction', '')}\n\n"
        f"### Input:\n"
        f"Question: {question}\n"
        f"Student Answer: {student_answer}\n"
        f"Model Answer: {model_answer}\n"
        f"Knowledge Base: {knowledge_base}\n"
        f"{rubric_text}\n"
        f"Max Score: {max_score}\n\n"
        f"### Response:\n{feedback}"
    )

    fixed_data.append({
        "text": formatted_text,
        "score": score  # Optional: Keep original score if needed
    })

# Save fixed file
with open("all_data_fixed.json", "w") as f:
    json.dump(fixed_data, f, indent=2)



In [19]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="all_data_fixed.json", split="train")
# dataset = dataset.map(formatting_prompts_func, batched=True)


In [24]:
import os

# Disable Triton line info to avoid compiler crash
os.environ["TRITON_DISABLE_LINE_INFO"] = "1"

from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",      # Ensure this field exists in your dataset
    max_seq_length=1024,            # Reduced for memory safety
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=1,        # Safer for memory
        gradient_accumulation_steps=8,        # Effective batch size = 8
        warmup_steps=5,
        num_train_epochs=1,
        max_steps=-1,                         # Use full dataset
        learning_rate=2e-4,
        fp16=True,                            # Use FP16 only
        bf16=False,                           # Disable BF16
        logging_steps=10,
        optim="adamw_8bit",                   # 8-bit optimizer
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",                     # No WandB/Hub reporting
    ),
)


In [25]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 960 | Num Epochs = 1 | Total steps = 120
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 167,772,160/8,000,000,000 (2.10% trained)


Step,Training Loss
10,0.075300
20,0.085600
30,0.086600
40,0.082700
50,0.079700
60,0.089900
70,0.117100
80,0.146000
90,0.367300
100,0.235700


In [30]:
from transformers import AutoTokenizer

alpaca_prompt = (
    "### Instruction:\n"
    "{instruction}\n\n"
    "### Input:\n"
    "Question: {question}\n"
    "Student Answer: {student_answer}\n"
    "Model Answer: {model_answer}\n"
    "Knowledge Base: {knowledge_base}\n"
    "Rubric: {rubric}\n"
    "Max Score: {max_score}\n\n"
    "### Response:\n"
)

filled_prompt = alpaca_prompt.format(
    instruction = "Grade the student answer based on model answer, knowledge base, and rubric. Provide a score and detailed feedback.",
    question = "Explain the role of primary keys in relational databases",
    # student_answer = "",  # Leave blank to let the model generate feedback
    # student_answer = "A primary key uniquely identifies each row in a database table and ensures that no duplicate records exist.",
    # student_answer = "Primary keys are used to find records in a table.",
    student_answer = "Primary keys are used to sort tables in alphabetical order.",



    model_answer = "A primary key is used to uniquely identify each record in a database table.",
    knowledge_base = "A primary key is used to uniquely identify each record in a database table.",
    rubric = "Conceptual Accuracy (40%), Technical Terminology Use (20%), Clarity and Structure (20%), Key Point Coverage (20%)",
    max_score = 4
)

# Tokenize and move to GPU
FastLanguageModel.for_inference(model)
inputs = tokenizer([filled_prompt], return_tensors="pt").to("cuda")

# Generate output
outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True)

# Decode output
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


### Instruction:
Grade the student answer based on model answer, knowledge base, and rubric. Provide a score and detailed feedback.

### Input:
Question: Explain the role of primary keys in relational databases
Student Answer: Primary keys are used to sort tables in alphabetical order.
Model Answer: A primary key is used to uniquely identify each record in a database table.
Knowledge Base: A primary key is used to uniquely identify each record in a database table.
Rubric: Conceptual Accuracy (40%), Technical Terminology Use (20%), Clarity and Structure (20%), Key Point Coverage (20%)
Max Score: 4

### Response:
Incorrect answer. Key concepts are missing or wrong.


In [31]:
# Save model (includes LoRA adapters if applicable)
trainer.model.save_pretrained("saved_model")

# Save tokenizer (important for decoding correctly later)
tokenizer.save_pretrained("saved_model")


('saved_model/tokenizer_config.json',
 'saved_model/special_tokens_map.json',
 'saved_model/chat_template.jinja',
 'saved_model/tokenizer.json')